In [ ]:
!pip install gensim


In [ ]:
import gensim
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_csv("/content/all_kindle_review.csv")

In [ ]:
data = data[["reviewText","rating"]]

In [ ]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [ ]:
data["sentiment"]=data["rating"].apply(lambda x:1 if x>=4 else 0 )


In [ ]:
import nltk
nltk.download('punkt_tab')
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', str(text).lower())  # Remove special characters and convert to lowercase
    words = word_tokenize(text)  # Tokenize
    words = [word for word in words if word not in stopwords.words('english')]  # Remove stopwords
    return words

In [ ]:
#apply preprocessing
data["reviewText"] = data["reviewText"].apply(preprocess_text)

In [ ]:
#train word2vec model
model = Word2Vec(sentences=data["reviewText"],vector_size=100,window=5,min_count=2,workers=4)


In [ ]:
#review into sentence embedding(avg work2vec)
import numpy as np
def sentence_vectors(tokens):
  vectors = [model.wv[word] for word in tokens if word in model.wv]
  return np.mean(vectors,axis=0) if vectors else np.zeros(100) #return zero vectr if word not found

In [ ]:
#apply tokens into vectors
data["vector"] = data["reviewText"].apply(sentence_vectors)

In [ ]:
 X= np.vstack(data["vector"].values)
 y=data["sentiment"].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
rf = RandomForestClassifier(n_estimators=100,random_state=42)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 0.76
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.77      0.76      1190
           1       0.77      0.75      0.76      1210

    accuracy                           0.76      2400
   macro avg       0.76      0.76      0.76      2400
weighted avg       0.76      0.76      0.76      2400



In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],         # Number of trees
    'max_depth': [10, 20, None],            # Maximum depth of trees
    'min_samples_split': [2, 5, 10],        # Minimum samples per split
    'min_samples_leaf': [1, 2, 5],          # Minimum samples per leaf
    'max_features': ['sqrt', 'log2']        # Features considered per split
}
grid_search = GridSearchCV(rf,param_grid,cv=5,scoring="accuracy",verbose=2)
grid_search.fit(X_train,y_train)
print(grid_search.best_estimator_)
y_pred_new = grid_search.predict(X_test)
# Evaluate the optimized model
accuracy = accuracy_score(y_test, y_pred)
print(f"Optimized Model Accuracy: {accuracy:.2f}")


Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.5s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.1s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.3s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.1s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.1s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   8.3s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   8.5s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; t